In [5]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras as extras


class BestPlayers:
    def __init__(self, order_by, query, column_names):
        self.order_by = order_by
        self.query = query
        self.column_names = column_names
        self.best_15 = self.execute_query()
        self.goalkeepers = [1]
        self.defenders = [3,4,5]
        self.midfielders = [3,4,5]
        self.attackers = [1,2,3]
        self.playing_goalkeeper = pd.DataFrame()
        self.playing_defenders = {}
        self.playing_midfielders ={}
        self.playing_attackers = {}


    def execute_query(self):
        conn = psycopg2.connect("dbname=fantasy_football user=postgres password=ca*m<>bL45g")
        cur = conn.cursor()
        cur.execute(self.query.format(by = self.order_by))
        d = cur.fetchall()
        cur.close()
        conn.close()
        df = pd.DataFrame(d)
        df.columns = self.column_names 
        return df
    
    def best_starting(self):
        squad_one_gkp = self.best_15[self.best_15['player_position'] == 'GKP'].\
                            sort_values(by = self.order_by, ascending=False).head(self.goalkeepers[0])
        self.playing_goalkeeper = squad_one_gkp

        poss_defenders = {i: self.best_15[self.best_15['player_position'] == 'DEF'].\
                            sort_values(by = self.order_by, ascending=False).head(i) for i in self.defenders} 
        self.playing_defenders = poss_defenders

        poss_midfielders = {i: self.best_15[self.best_15['player_position'] == 'MID'].\
                            sort_values(by = self.order_by, ascending=False).head(i) for i in self.midfielders}
        self.playing_midfielders = poss_midfielders

        poss_attackers = {i: self.best_15[self.best_15['player_position'] == 'FWD'].\
                            sort_values(by = self.order_by, ascending=False).head(i) for i in self.attackers}
        self.playing_forwards = poss_attackers

    def top_per_position(self):
        self.execute_query()
        top_goal = self.best_15[self.best_15['player_position'] == 'GKP'].\
                            sort_values(by = self.order_by, ascending=False).head(10)

        top_defs = self.best_15[self.best_15['player_position'] == 'DEF'].\
                            sort_values(by = self.order_by, ascending=False).head(10) 


        top_mids = self.best_15[self.best_15['player_position'] == 'MID'].\
                            sort_values(by = self.order_by, ascending=False).head(10)


        top_fwds = self.best_15[self.best_15['player_position'] == 'FWD'].\
                            sort_values(by = self.order_by, ascending=False).head(10)
    def run_process(self, formations_):
        self.execute_query()
        self.best_starting()
        optimal = {}
        for i in formations_:
            goal = self.playing_goalkeeper
            defender = self.playing_defenders[i[1]]
            midfielder = self.playing_midfielders[i[2]]
            forward = self.playing_forwards[i[3]]
            frames = [goal, defender, midfielder, forward]
            df_formation = pd.concat(frames)
            total = df_formation[self.order_by].sum()
            optimal[float(total)] = df_formation
        return optimal[max(optimal.keys())]

    


        

In [266]:


query = """
    
 SELECT

*

FROM
(

SELECT 

    A.*
    ,row_number() over(partition by player_position order by {by} desc) as ranked_position_points

    FROM(

    SELECT 
        first_name
        ,second_name
        ,web_name
        ,t1.id_
        ,now_cost
        ,points_per_game
        ,selected_by_percent
        ,team
        ,team_code
        ,total_points
        ,minutes
        ,goals_scored
        ,assists
        ,clean_sheets
        ,goals_conceded
        ,yellow_cards
        ,red_cards
        ,bonus
        ,cost_change_start
        ,cost_change_event
        ,dreamteam_count
        ,selected_by_percent
        ,t2.plural_name_short as player_position
        ,squad_select
        ,squad_min_play
        ,squad_max_play
        ,cast(cast(total_points as decimal(20,2))/cast(now_cost as decimal(20,2)) as decimal(10,2)) 
            AS points_per_cost
 
        

    FROM fantasy_football t1

    INNER JOIN player_types t2
        ON t1.element_type = t2.id_) A)B
        
    WHERE CASE WHEN SQUAD_SELECT = 2 THEN B.ranked_position_points IN (1,2)
                WHEN SQUAD_SELECT = 5 THEN B.ranked_position_points IN (1,2,3,4,5)
                WHEN SQUAD_SELECT = 3 THEN B.ranked_position_points IN (1,2,3) END 
    
    
        
   
    
    """

column_names = [
                'first_name',
                'second_name',
                'web_name',
                't1.id_',
                'now_cost',
                'points_per_game',
                'selected_by_percent',
                'team',
                'team_code',
                'total_points',
                'minutes',
                'goals_scored',
                'assists',
                'clean_sheets',
                'goals_conceded',
                'yellow_cards',
                'red_cards',
                'bonus',
                'cost_change_start',
                'cost_change_event',
                'dreamteam_count',
                'selected_by_percent',
                'player_position',
                'squad_select',
                'squad_min_play',
                'squad_max_play',
                'points_per_cost',
                'ranked_position_points']


try:
    squad = BestPlayers(order_by = 'points_per_cost', query = query, column_names =column_names).execute_query()
    if len(squad) == 15:
        print('Returned 15 Players')
    else:
        print('Did not return correct number of rows')

except:
    print('Class failed to import ')



Returned 15 Players


In [271]:
run = BestPlayers(order_by = 'points_per_cost', query = query, column_names =column_names)

In [156]:
formations = [[1,3,4,3], [1,3,5,2], [1,4,4,2], [1,4,5,1], [1,4,3,3], [1,5,3,2], [1,5,4,1]]

In [273]:
run.execute_query()

,first_name,second_name,web_name,t1.id_,now_cost,points_per_game,selected_by_percent,team,team_code,total_points,...,cost_change_start,cost_change_event,dreamteam_count,selected_by_percent,player_position,squad_select,squad_min_play,squad_max_play,points_per_cost,ranked_position_points
0,Nathan,Collins,Collins,516,45,4.7,1.1,20,39,28,...,0,0,1,1.1,DEF,5,3,5,0.62,1
1,William,Saliba,Saliba,26,48,4.8,21.1,1,3,29,...,3,0,1,21.1,DEF,5,3,5,0.60,2
2,Max,Kilman,Kilman,484,45,4.5,1.4,20,39,27,...,0,0,1,1.4,DEF,5,3,5,0.60,3
3,Kieran,Trippier,Trippier,357,52,5.0,33.7,15,4,30,...,2,1,0,33.7,DEF,5,3,5,0.58,4
4,Fabian,Schär,Schär,366,47,5.4,7.7,15,4,27,...,2,0,1,7.7,DEF,5,3,5,0.57,5
5,Ivan,Toney,Toney,80,72,7.5,19.0,4,94,45,...,2,1,1,19.0,FWD,3,1,3,0.63,1
6,Aleksandar,Mitrović,Mitrović,210,68,6.5,24.9,9,54,39,...,3,0,1,24.9,FWD,3,1,3,0.57,2
7,Erling,Haaland,Haaland,318,120,11.2,78.5,13,43,67,...,5,1,3,78.5,FWD,3,1,3,0.56,3
8,Nick,Pope,Pope,376,51,6.0,10.7,15,4,36,...,1,1,1,10.7,GKP,2,1,1,0.71,1
9,José,Malheiro de Sá,Sá,478,50,5.8,5.5,20,39,35,...,0,0,1,5.5,GKP,2,1,1,0.70,2


### Say Code starts from here

In [5]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras as extras

In [6]:
query_2 = '''

 
 SELECT

*

FROM
(

SELECT 

    A.*
    ,row_number() over(partition by player_position order by points_per_cost desc) as ranked_position_points
    ,row_number() over(partition by team order by points_per_cost desc) as ranked_team_points

    FROM(

    SELECT 
        first_name
        ,second_name
        ,web_name
        ,t1.id_
        ,now_cost
        ,points_per_game
        ,selected_by_percent
        ,team
        ,team_code
        ,total_points
        ,minutes
        ,goals_scored
        ,assists
        ,clean_sheets
        ,goals_conceded
        ,yellow_cards
        ,red_cards
        ,bonus
        ,cost_change_start
        ,cost_change_event
        ,dreamteam_count
        ,selected_by_percent
        ,t2.plural_name_short as player_position
        ,squad_select
        ,squad_min_play
        ,squad_max_play
        ,cast(cast(total_points as decimal(20,2))/cast(now_cost as decimal(20,2)) as decimal(10,2)) 
            AS points_per_cost
 
        

    FROM fantasy_football t1

    INNER JOIN player_types t2
        ON t1.element_type = t2.id_) A)B
        

'''



In [7]:
column_names_demo = [
                'first_name',
                'second_name',
                'web_name',
                't1.id_',
                'now_cost',
                'points_per_game',
                'selected_by_percent',
                'team',
                'team_code',
                'total_points',
                'minutes',
                'goals_scored',
                'assists',
                'clean_sheets',
                'goals_conceded',
                'yellow_cards',
                'red_cards',
                'bonus',
                'cost_change_start',
                'cost_change_event',
                'dreamteam_count',
                'selected_by_percent',
                'player_position',
                'squad_select',
                'squad_min_play',
                'squad_max_play',
                'points_per_cost',
                'ranked_position_points',
                'ranked_team_points']

In [8]:
def execute_query(query, names):
    conn = psycopg2.connect("dbname=fantasy_football user=postgres password=ca*m<>bL45g")
    cur = conn.cursor()
    cur.execute(query)
    d = cur.fetchall()
    cur.close()
    conn.close()
    df = pd.DataFrame(d)
    df.columns = column_names_demo
    return df

In [9]:
total_df = execute_query(query_2, column_names_demo)

In [34]:
total_df

,first_name,second_name,web_name,t1.id_,now_cost,points_per_game,selected_by_percent,team,team_code,total_points,...,cost_change_event,dreamteam_count,selected_by_percent,player_position,squad_select,squad_min_play,squad_max_play,points_per_cost,ranked_position_points,ranked_team_points
0,Nathan,Collins,Collins,516,45,4.7,1.1,20,39,28,...,0,1,1.1,DEF,5,3,5,0.62,1,2
1,William,Saliba,Saliba,26,48,4.8,21.1,1,3,29,...,0,1,21.1,DEF,5,3,5,0.60,2,1
2,Max,Kilman,Kilman,484,45,4.5,1.4,20,39,27,...,0,1,1.4,DEF,5,3,5,0.60,3,3
3,Kieran,Trippier,Trippier,357,52,5.0,33.7,15,4,30,...,1,0,33.7,DEF,5,3,5,0.58,4,2
4,Joël,Veltman,Veltman,109,46,4.3,5.6,5,36,26,...,0,2,5.6,DEF,5,3,5,0.57,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,Archie,Gray,Gray,565,45,0.0,0.1,11,2,0,...,0,0,0.1,MID,5,2,5,0.00,266,27
620,Raphael,Dias Belloli,Raphinha,232,70,0.0,0.1,11,2,0,...,0,0,0.1,MID,5,2,5,0.00,267,26
621,David,Brooks,Brooks,60,53,0.0,0.0,3,91,0,...,0,0,0.0,MID,5,2,5,0.00,268,26
622,Lewis,Bate,Bate,241,45,0.0,0.0,11,2,0,...,0,0,0.0,MID,5,2,5,0.00,269,23


In [55]:
from pulp import *
prob = pulp.LpProblem('FantasyTeam', LpMaximize)

In [56]:
decision_variables = []
for rownum, row in total_df.iterrows():
    variable = str('x' + str(rownum))
    variable = pulp.LpVariable(str(variable), lowBound = 0, upBound = 1, cat= 'Integer') #make variables binary
    decision_variables.append(variable)

print ("Total number of decision_variables: " + str(len(decision_variables)))

Total number of decision_variables: 624


In [57]:
total_points = ""
for rownum, row in total_df.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            formula = row['total_points']*player
            total_points += formula

prob += total_points
display ("Optimization function: " + str(total_points))

'Optimization function: 28*x0 + 29*x1 + 23*x10 + 3*x100 + 3*x101 + 3*x102 + 3*x103 + 3*x104 + 3*x105 + 2*x106 + 2*x107 + 2*x108 + 2*x109 + 23*x11 + 2*x110 + 2*x111 + 2*x112 + 2*x113 + 2*x114 + 2*x115 + 2*x116 + 2*x117 + 2*x118 + 2*x119 + 22*x12 + x120 + x121 + 2*x122 + x123 + x124 + x125 + x126 + x127 + x128 + x129 + 22*x13 + x130 + x131 + x132 + x133 + x134 + x135 + 21*x14 + 33*x15 + 25*x16 + 22*x17 + 19*x18 + 19*x19 + 27*x2 + 19*x20 + 20*x21 - x219 + 19*x22 - x220 - x221 - x222 + 45*x223 + 39*x224 + 67*x225 + 35*x226 + 33*x227 + 22*x228 + 20*x229 + 19*x23 + 21*x230 + 40*x231 + 22*x232 + 24*x233 + 20*x234 + 17*x235 + 14*x236 + 14*x237 + 14*x238 + 15*x239 + 18*x24 + 16*x240 + 13*x241 + 10*x242 + 14*x243 + 17*x244 + 14*x245 + 9*x246 + 11*x247 + 7*x248 + 7*x249 + 17*x25 + 11*x250 + 9*x251 + 7*x252 + 8*x253 + 9*x254 + 5*x255 + 6*x256 + 4*x257 + 4*x258 + 5*x259 + 20*x26 + 5*x260 + 3*x261 + 7*x262 + 3*x263 + 3*x264 + 4*x265 + 3*x266 + 2*x267 + 2*x268 + 2*x269 + 17*x27 + x270 + x271 + x272 +

In [58]:
avail_cash = 1000
total_paid = ""
for rownum, row in total_df.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            formula = row['now_cost']*player
            total_paid += formula

prob += (total_paid <= avail_cash)

In [59]:
prob

FantasyTeam:
MAXIMIZE
28*x0 + 29*x1 + 23*x10 + 3*x100 + 3*x101 + 3*x102 + 3*x103 + 3*x104 + 3*x105 + 2*x106 + 2*x107 + 2*x108 + 2*x109 + 23*x11 + 2*x110 + 2*x111 + 2*x112 + 2*x113 + 2*x114 + 2*x115 + 2*x116 + 2*x117 + 2*x118 + 2*x119 + 22*x12 + 1*x120 + 1*x121 + 2*x122 + 1*x123 + 1*x124 + 1*x125 + 1*x126 + 1*x127 + 1*x128 + 1*x129 + 22*x13 + 1*x130 + 1*x131 + 1*x132 + 1*x133 + 1*x134 + 1*x135 + 21*x14 + 33*x15 + 25*x16 + 22*x17 + 19*x18 + 19*x19 + 27*x2 + 19*x20 + 20*x21 + -1*x219 + 19*x22 + -1*x220 + -1*x221 + -1*x222 + 45*x223 + 39*x224 + 67*x225 + 35*x226 + 33*x227 + 22*x228 + 20*x229 + 19*x23 + 21*x230 + 40*x231 + 22*x232 + 24*x233 + 20*x234 + 17*x235 + 14*x236 + 14*x237 + 14*x238 + 15*x239 + 18*x24 + 16*x240 + 13*x241 + 10*x242 + 14*x243 + 17*x244 + 14*x245 + 9*x246 + 11*x247 + 7*x248 + 7*x249 + 17*x25 + 11*x250 + 9*x251 + 7*x252 + 8*x253 + 9*x254 + 5*x255 + 6*x256 + 4*x257 + 4*x258 + 5*x259 + 20*x26 + 5*x260 + 3*x261 + 7*x262 + 3*x263 + 3*x264 + 4*x265 + 3*x266 + 2*x267 + 2*x268 

In [60]:
avail_gk = 2
total_gk = ""
for rownum, row in total_df.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            if row['player_position'] == 'GKP':
                formula = 1*player
                total_gk += formula
prob += (total_gk == avail_gk)
print(total_gk)

x292 + x293 + x294 + x295 + x296 + x297 + x298 + x299 + x300 + x301 + x302 + x303 + x304 + x305 + x306 + x307 + x308 + x309 + x310 + x311 + x312 + x313 + x314 + x315 + x316 + x317 + x318 + x319 + x320 + x321 + x322 + x323 + x324 + x325 + x326 + x327 + x328 + x329 + x330 + x331 + x332 + x333 + x334 + x335 + x336 + x337 + x338 + x339 + x340 + x341 + x342 + x343 + x344 + x345 + x346 + x347 + x348 + x349 + x350 + x351 + x352 + x353


In [61]:
avail_def = 5
total_def = ""
for rownum, row in total_df.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            if row['player_position'] == 'DEF':
                formula = 1*player
                total_def += formula
prob += (total_def == avail_def)
print((total_def))

x0 + x1 + x10 + x100 + x101 + x102 + x103 + x104 + x105 + x106 + x107 + x108 + x109 + x11 + x110 + x111 + x112 + x113 + x114 + x115 + x116 + x117 + x118 + x119 + x12 + x120 + x121 + x122 + x123 + x124 + x125 + x126 + x127 + x128 + x129 + x13 + x130 + x131 + x132 + x133 + x134 + x135 + x136 + x137 + x138 + x139 + x14 + x140 + x141 + x142 + x143 + x144 + x145 + x146 + x147 + x148 + x149 + x15 + x150 + x151 + x152 + x153 + x154 + x155 + x156 + x157 + x158 + x159 + x16 + x160 + x161 + x162 + x163 + x164 + x165 + x166 + x167 + x168 + x169 + x17 + x170 + x171 + x172 + x173 + x174 + x175 + x176 + x177 + x178 + x179 + x18 + x180 + x181 + x182 + x183 + x184 + x185 + x186 + x187 + x188 + x189 + x19 + x190 + x191 + x192 + x193 + x194 + x195 + x196 + x197 + x198 + x199 + x2 + x20 + x200 + x201 + x202 + x203 + x204 + x205 + x206 + x207 + x208 + x209 + x21 + x210 + x211 + x212 + x213 + x214 + x215 + x216 + x217 + x218 + x219 + x22 + x220 + x221 + x222 + x23 + x24 + x25 + x26 + x27 + x28 + x29 + x3 +

In [62]:
avail_mid = 5
total_mid = ""
for rownum, row in total_df.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            if row['player_position'] == 'MID':
                formula = 1*player
                total_mid += formula
prob += (total_mid == avail_mid)
print((total_mid))

x354 + x355 + x356 + x357 + x358 + x359 + x360 + x361 + x362 + x363 + x364 + x365 + x366 + x367 + x368 + x369 + x370 + x371 + x372 + x373 + x374 + x375 + x376 + x377 + x378 + x379 + x380 + x381 + x382 + x383 + x384 + x385 + x386 + x387 + x388 + x389 + x390 + x391 + x392 + x393 + x394 + x395 + x396 + x397 + x398 + x399 + x400 + x401 + x402 + x403 + x404 + x405 + x406 + x407 + x408 + x409 + x410 + x411 + x412 + x413 + x414 + x415 + x416 + x417 + x418 + x419 + x420 + x421 + x422 + x423 + x424 + x425 + x426 + x427 + x428 + x429 + x430 + x431 + x432 + x433 + x434 + x435 + x436 + x437 + x438 + x439 + x440 + x441 + x442 + x443 + x444 + x445 + x446 + x447 + x448 + x449 + x450 + x451 + x452 + x453 + x454 + x455 + x456 + x457 + x458 + x459 + x460 + x461 + x462 + x463 + x464 + x465 + x466 + x467 + x468 + x469 + x470 + x471 + x472 + x473 + x474 + x475 + x476 + x477 + x478 + x479 + x480 + x481 + x482 + x483 + x484 + x485 + x486 + x487 + x488 + x489 + x490 + x491 + x492 + x493 + x494 + x495 + x496 +

In [63]:
avail_fwd = 3
total_fwd = ""
for rownum, row in total_df.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            if row['player_position'] == 'FWD':
                formula = 1*player
                total_fwd += formula
prob += (total_fwd == avail_fwd)
print(total_fwd)

x223 + x224 + x225 + x226 + x227 + x228 + x229 + x230 + x231 + x232 + x233 + x234 + x235 + x236 + x237 + x238 + x239 + x240 + x241 + x242 + x243 + x244 + x245 + x246 + x247 + x248 + x249 + x250 + x251 + x252 + x253 + x254 + x255 + x256 + x257 + x258 + x259 + x260 + x261 + x262 + x263 + x264 + x265 + x266 + x267 + x268 + x269 + x270 + x271 + x272 + x273 + x274 + x275 + x276 + x277 + x278 + x279 + x280 + x281 + x282 + x283 + x284 + x285 + x286 + x287 + x288 + x289 + x290 + x291


In [64]:
team_dict= {}
for team in set(total_df.team):
    team_dict[str(team)]=dict()
    team_dict[str(team)]['avail'] = 3
    team_dict[str(team)]['total'] = ""
    for rownum, row in total_df.iterrows():
        for i, player in enumerate(decision_variables):
            if rownum == i:
                if row['team'] == team:
                    formula = 1*player
                    team_dict[str(team)]['total'] += formula

    prob += (team_dict[str(team)]['total'] <= team_dict[str(team)]['avail'])
    print(len(team_dict))
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [67]:
prob

FantasyTeam:
MAXIMIZE
28*x0 + 29*x1 + 23*x10 + 3*x100 + 3*x101 + 3*x102 + 3*x103 + 3*x104 + 3*x105 + 2*x106 + 2*x107 + 2*x108 + 2*x109 + 23*x11 + 2*x110 + 2*x111 + 2*x112 + 2*x113 + 2*x114 + 2*x115 + 2*x116 + 2*x117 + 2*x118 + 2*x119 + 22*x12 + 1*x120 + 1*x121 + 2*x122 + 1*x123 + 1*x124 + 1*x125 + 1*x126 + 1*x127 + 1*x128 + 1*x129 + 22*x13 + 1*x130 + 1*x131 + 1*x132 + 1*x133 + 1*x134 + 1*x135 + 21*x14 + 33*x15 + 25*x16 + 22*x17 + 19*x18 + 19*x19 + 27*x2 + 19*x20 + 20*x21 + -1*x219 + 19*x22 + -1*x220 + -1*x221 + -1*x222 + 45*x223 + 39*x224 + 67*x225 + 35*x226 + 33*x227 + 22*x228 + 20*x229 + 19*x23 + 21*x230 + 40*x231 + 22*x232 + 24*x233 + 20*x234 + 17*x235 + 14*x236 + 14*x237 + 14*x238 + 15*x239 + 18*x24 + 16*x240 + 13*x241 + 10*x242 + 14*x243 + 17*x244 + 14*x245 + 9*x246 + 11*x247 + 7*x248 + 7*x249 + 17*x25 + 11*x250 + 9*x251 + 7*x252 + 8*x253 + 9*x254 + 5*x255 + 6*x256 + 4*x257 + 4*x258 + 5*x259 + 20*x26 + 5*x260 + 3*x261 + 7*x262 + 3*x263 + 3*x264 + 4*x265 + 3*x266 + 2*x267 + 2*x268 

In [68]:
prob.writeLP('FantasyTeam.lp')
optimization_result = prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/fraserbrown/fantasy_football/.venv/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/k3/lq_kt5093pz8d_q34rwznsyc0000gn/T/c25f5e1c51ec4411bb261ad2526aae18-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/k3/lq_kt5093pz8d_q34rwznsyc0000gn/T/c25f5e1c51ec4411bb261ad2526aae18-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 3550 RHS
At line 3576 BOUNDS
At line 4201 ENDATA
Problem MODEL has 25 rows, 624 columns and 1872 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 566.087 - 0.00 seconds
Cgl0004I processed model has 25 rows, 552 columns (552 integer (498 of which binary)) and 1656 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.173913
Cbc0038I Solu

In [69]:
variable_name = []
variable_value = []

for v in prob.variables():
    variable_name.append(v.name)
    variable_value.append(v.varValue)

df = pd.DataFrame({'variable': variable_name, 'value': variable_value})
for rownum, row in df.iterrows():
    value = re.findall(r'(\d+)', row['variable'])
    df.loc[rownum, 'variable'] = int(value[0])

df = df.sort_index()

In [70]:
df

,variable,value
0,0,1.0
1,1,1.0
2,10,0.0
3,100,0.0
4,101,0.0
...,...,...
619,95,0.0
620,96,0.0
621,97,0.0
622,98,0.0


In [71]:
variable_name = []
variable_value = []

for v in prob.variables():
    variable_name.append(v.name)
    variable_value.append(v.varValue)

df = pd.DataFrame({'variable': variable_name, 'value': variable_value})
for rownum, row in df.iterrows():
    value = re.findall(r'(\d+)', row['variable'])
    df.loc[rownum, 'variable'] = int(value[0])

df = df.sort_index()

#append results
for rownum, row in total_df.iterrows():
    for results_rownum, results_row in df.iterrows():
        if rownum == results_row['variable']:
            total_df.loc[rownum, 'decision'] = results_row['value']

total_df[total_df.decision==1].now_cost.sum() 
total_df[total_df.decision==1].total_points.sum() 
best_squad = total_df[total_df.decision==1].sort_values('player_position').head(15)

In [1]:
best_squad['total_points'].sum()

NameError: name 'best_squad' is not defined

In [10]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras as extras
from pulp import *

class BestPlayers:
    def __init__(self, order_by, query, column_names):
        self.order_by = order_by
        self.query = query
        self.column_names = column_names
        self.df = self.execute_query()
        self.prob= self.create_problem_variable()
        self.decision_variables = []
        self.optimisation_result = self.solve_optimisation_problem()


    def execute_query(self):
        conn = psycopg2.connect("dbname=fantasy_football user=postgres password=ca*m<>bL45g")
        cur = conn.cursor()
        cur.execute(self.query.format(by = self.order_by))
        d = cur.fetchall()
        cur.close()
        conn.close()
        df = pd.DataFrame(d)
        df.columns = self.column_names 
        return df

    def create_problem_variable(self):
        self.prob = pulp.LpProblem('FantasyTeam', LpMaximize)
        

    def create_decision_variables(self):
        #self.decision_variables = []
        for rownum, row in self.df.iterrows():
            variable = str('x' + str(rownum))
            variable = pulp.LpVariable(str(variable), lowBound = 0, upBound = 1, cat= 'Integer') #make variables binary
            self.decision_variables.append(variable)

    def create_optimisation_problem(self):
        self.create_problem_variable()
        self.create_decision_variables()
        total_points = ""
        for rownum, row in self.df.iterrows():
            for i, player in enumerate(self.decision_variables):
                if rownum == i:
                    formula = row['total_points']*player
                    total_points += formula

        self.prob += total_points

        avail_gk = 2
        total_gk = ""
        for rownum, row in self.df.iterrows():
            for i, player in enumerate(self.decision_variables):
                if rownum == i:
                    if row['player_position'] == 'GKP':
                        formula = 1*player
                        total_gk += formula
        self.prob += (total_gk == avail_gk)

        avail_mid = 5
        total_mid = ""
        for rownum, row in self.df.iterrows():
            for i, player in enumerate(self.decision_variables):
                if rownum == i:
                    if row['player_position'] == 'MID':
                        formula = 1*player
                        total_mid += formula
        self.prob += (total_mid == avail_mid)

        avail_def = 5
        total_def = ""
        for rownum, row in self.df.iterrows():
            for i, player in enumerate(self.decision_variables):
                if rownum == i:
                    if row['player_position'] == 'DEF':
                        formula = 1*player
                        total_def += formula
        self.prob += (total_def == avail_def)


        avail_fwd = 3
        total_fwd = ""
        for rownum, row in self.df.iterrows():
            for i, player in enumerate(self.decision_variables):
                if rownum == i:
                    if row['player_position'] == 'FWD':
                        formula = 1*player
                        total_fwd += formula
        self.prob += (total_fwd == avail_fwd)

        avail_cash = 1000
        total_paid = ""
        for rownum, row in self.df.iterrows():
            for i, player in enumerate(self.decision_variables):
                if rownum == i:
                    formula = row['now_cost']*player
                    total_paid += formula

        self.prob += (total_paid <= avail_cash)

        team_dict= {}
        for team in set(self.df.team):
            team_dict[str(team)]=dict()
            team_dict[str(team)]['avail'] = 3
            team_dict[str(team)]['total'] = ""
            for rownum, row in self.df.iterrows():
                for i, player in enumerate(self.decision_variables):
                    if rownum == i:
                        if row['team'] == team:
                            formula = 1*player
                            team_dict[str(team)]['total'] += formula

            self.prob += (team_dict[str(team)]['total'] <= team_dict[str(team)]['avail'])


    def solve_optimisation_problem(self):
        self.create_optimisation_problem()
        self.prob.writeLP('FantasyTeam.lp')
        optimization_result = self.prob.solve()
        self.optimisation_result = optimization_result 


    def return_team(self):
        self.solve_optimisation_problem()
        variable_name = []
        variable_value = []

        for v in self.prob.variables():
            variable_name.append(v.name)
            variable_value.append(v.varValue)

        data = pd.DataFrame({'variable': variable_name, 'value': variable_value})
        for rownum, row in data.iterrows():
            value = re.findall(r'(\d+)', row['variable'])
            data.loc[rownum, 'variable'] = int(value[0])

        data = data.sort_index()

        #append results
        for rownum, row in self.df.iterrows():
            for results_rownum, results_row in data.iterrows():
                if rownum == results_row['variable']:
                    self.df.loc[rownum, 'decision'] = results_row['value']

        self.df[self.df.decision==1].now_cost.sum() 
        self.df[self.df.decision==1].total_points.sum() 
        best_squad = self.df[self.df.decision==1].sort_values('player_position').head(15)
        return best_squad


    

            

        

In [11]:
df= BestPlayers(order_by = 'total_points', query = query_2, column_names =column_names_demo).return_team()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/fraserbrown/fantasy_football/.venv/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/k3/lq_kt5093pz8d_q34rwznsyc0000gn/T/a8972655b47640669108a3ed968d9c4a-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/k3/lq_kt5093pz8d_q34rwznsyc0000gn/T/a8972655b47640669108a3ed968d9c4a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 3550 RHS
At line 3576 BOUNDS
At line 4201 ENDATA
Problem MODEL has 25 rows, 624 columns and 1872 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 566.087 - 0.00 seconds
Cgl0004I processed model has 25 rows, 551 columns (551 integer (496 of which binary)) and 1653 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.173913
Cbc0038I Solu

In [13]:
df

,first_name,second_name,web_name,t1.id_,now_cost,points_per_game,selected_by_percent,team,team_code,total_points,...,dreamteam_count,selected_by_percent,player_position,squad_select,squad_min_play,squad_max_play,points_per_cost,ranked_position_points,ranked_team_points,decision
0,Nathan,Collins,Collins,516,45,4.7,1.1,20,39,28,...,1,1.1,DEF,5,3,5,0.62,1,2,1.0
1,William,Saliba,Saliba,26,48,4.8,21.1,1,3,29,...,1,21.1,DEF,5,3,5,0.60,2,1,1.0
2,Max,Kilman,Kilman,484,45,4.5,1.4,20,39,27,...,1,1.4,DEF,5,3,5,0.60,3,3,1.0
3,Kieran,Trippier,Trippier,357,52,5.0,33.7,15,4,30,...,0,33.7,DEF,5,3,5,0.58,4,2,1.0
15,João,Cancelo,Cancelo,306,71,5.5,47.9,13,43,33,...,2,47.9,DEF,5,3,5,0.46,16,3,1.0
223,Ivan,Toney,Toney,80,72,7.5,19.0,4,94,45,...,1,19.0,FWD,3,1,3,0.63,1,1,1.0
224,Aleksandar,Mitrović,Mitrović,210,68,6.5,24.9,9,54,39,...,1,24.9,FWD,3,1,3,0.57,2,1,1.0
225,Erling,Haaland,Haaland,318,120,11.2,78.5,13,43,67,...,3,78.5,FWD,3,1,3,0.56,3,1,1.0
292,Nick,Pope,Pope,376,51,6.0,10.7,15,4,36,...,1,10.7,GKP,2,1,1,0.71,1,1,1.0
293,José,Malheiro de Sá,Sá,478,50,5.8,5.5,20,39,35,...,1,5.5,GKP,2,1,1,0.70,2,1,1.0


In [337]:
print('Number of points:' + str(df['total_points'].sum()) + ' , ' + 'Cost of Squad:' + '£' + str(df['now_cost'].sum()/10) + 'M')

Number of points:566 , Cost of Squad:£99.6M


In [ ]:
avail_gk = 2
total_gk = ""
for rownum, row in self.df.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            if row['player_position'] == 'GKP':
                formula = 1*player
                total_gk += formula
self.prob += (total_gk == avail_gk)

avail_mid = 5
total_mid = ""
for rownum, row in self.df.iterrows():
    for i, player in enumerate(decision_variables):       ###### The players could be made into a for loop
        if rownum == i:
            if row['player_position'] == 'MID':
                formula = 1*player
                total_mid += formula
self.prob += (total_mid == avail_mid)

avail_def = 5
total_def = ""
for rownum, row in self.df.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            if row['player_position'] == 'DEF':
                formula = 1*player
                total_def += formula
self.prob += (total_def == avail_def)


avail_fwd = 3
total_fwd = ""
for rownum, row in self.df.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            if row['player_position'] == 'FWD':
                formula = 1*player
                total_fwd += formula
self.prob += (total_fwd == avail_fwd)